In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# marginal utility and capital

def u_prime(c, γ=1.5):
    '''
    Utility function
    '''
    res = c**-γ
    
    # for nan values
    res[np.isnan(res)] = 0
    
    return res


for i, y in enumerate(y_grid):
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4), sharex=True)
    
    fig.suptitle(f'$y_t = {y}$')

    # utility
    ax1.plot(k_grid_1, u(consumption_1[i]), label='$a_t \geq 0.01$',
             c=colors[0], alpha=0.6, lw=3)
    ax1.plot(k_grid_2, u(consumption_2[i]), label='$a_t \geq 2.0$',
             c=colors[1], alpha=0.6, lw=3)

    ax1.set_title('Utility')
    ax1.set_xlabel('$a_t$')
    ax1.set_ylabel('$u(c_t)$')
    ax1.legend()
    
    # marginal utility
    ax2.plot(k_grid_1, u_prime(consumption_1[i]), label='$a_t \geq 0.01$',
             c=colors[0],  alpha=0.6, lw=3)
    ax2.plot(k_grid_2, u_prime(consumption_2[i]), label='$a_t \geq 2.0$',
             c=colors[1],  alpha=0.6, lw=3)

    ax2.set_title('Marginal Utility')
    ax2.set_xlabel('$a_t$')
    ax2.set_ylabel('$u\'(c_t)$')
    ax2.legend()

    plt.show()

In [6]:
β = 0.96
y_grid = np.array([1.5, 1, 0.5])
n = len(y_grid)

π = np.array([
    [0.5, 0.25, 0.25],
    [0.25, 0.5, 0.25],
    [0.25, 0.25, 0.5]
])


In [48]:
# stochastic discount factor

m = np.zeros((n, n))

for i in range(n):
    for j in range(n):
        
        # m[c, c'] = u'(c)/u'(c')
        m[i, j] = y_grid[i] / y_grid[j] # derivative of ln(x) = 1/x
        
print(m)

[[1.         1.5        3.        ]
 [0.66666667 1.         2.        ]
 [0.33333333 0.5        1.        ]]


In [49]:
T = 1000
P = np.zeros((T, n))

for i in range(T-1):
    
    for j in range(n):
        
        # β*E[m(P' + y')]
        P[i + 1, j] = β * np.dot(π[j], m[j] * (y_grid + P[i]))
    
    # convergence
    if np.max(np.abs(P[i + 1, :] - P[i, :])) < 1e-10:
        print(f"Convergence after {i} periods")
        P_converge = P[i]
        break

Convergence after 573 periods


In [50]:
P_converge

array([36., 24., 12.])

In [51]:
coeff = np.eye(n) - β * π * m
RHS = β * np.dot(π * m, y_grid)
np.linalg.solve(coeff, RHS)

array([36., 24., 12.])

In [62]:
def asset_price(β, y_grid, π, γ):
    
    n = len(y_grid)
    
    m = np.zeros((n, n))

    for i in range(n):
        for j in range(n):

            # m = (u'(c')/u'(c))^-γ
            m[i, j] = (y_grid[j] / y_grid[i]) ** -γ # derivative of ln(x) = 1/x

    
    P = np.zeros(n)

    i = 0
    while True:

        P_prime = P.copy()

        for j in range(n):

            # β * E[m * (P' + y')]
            P[j] = β * np.dot(π[j], m[j] * (y_grid + P_prime))

        i += 1
        # check convergence
        if all(P == P_prime):
            
            print(f"Convergence after {i} periods")
            
            return P

In [65]:
print(asset_price(β, y_grid, π, 1))
print(asset_price(β, y_grid, π, 2))

Convergence after 829 periods
[36. 24. 12.]
Convergence after 825 periods
[65.60526316 29.26315789  7.39473684]


In [67]:
π_new = np.array([[0.7, 0.15, 0.15], [0.15, 0.7, 0.15], [0.15, 0.15, 0.7]])

print(asset_price(β, y_grid, π_new, 1))
print(asset_price(β, y_grid, π_new, 2))

Convergence after 820 periods
[36. 24. 12.]
Convergence after 817 periods
[64.60169492 29.08474576  7.55084746]


In [68]:
y_grid_new = [1.8, 1.0, 0.2]

print(asset_price(β, y_grid_new, π_new, 1))
print(asset_price(β, y_grid_new, π_new, 2))

Convergence after 817 periods
[43.2 24.   4.8]
Convergence after 810 periods
[164.01355932  51.11864407   2.22372881]
